In [3]:
!pip install ptan
!pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/38/d2/3e8c13ffc37ca5ebc6f382b242b44acb43eb489042e1728407ac3904e72f/opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.1 MB 3.2 MB/s eta 0:00:12
   ---------------------------------------- 0.3/38.1 MB 4.3 MB/s eta 0:00:09
    --------------------------------------- 0.6/38.1 MB 5.3 MB/s eta 0:00:08
   - -------------------------------------- 1.1/38.1 MB 7.0 MB/s eta 0:00:06
   - -------------------------------------- 1.6/38.1 MB 8.1 MB/s eta 0:00:05
   --- ------------------------------------ 3.0/38.1 MB 12.0 MB/s eta 0:00:03
   --- ------------------------------------ 3.7/38.1 MB 11.9 MB/s eta 0:00:03
   ---- ----------------------------------- 4.1/38.1 MB 11.5 MB/s eta 0:00:03
   ----- ---------------------------------- 5.5/38.1 MB 13.4 MB/s eta 0:00:03
   ------ --

In [4]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from transformers import ViTFeatureExtractor, ViTForImageClassification,AutoFeatureExtractor,AdamW
from tqdm import tqdm
import torch
import numpy as np
from transformers import ViTImageProcessor, ViTModel
from PIL import Image

In [5]:
import ptan
import gym
import numpy as np
from typing import List, Any, Optional, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pylab as plt

%matplotlib inline

In [6]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    
    print("CUDA is available. Using GPU.")
else:
    
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU.


In [ ]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k')
# inputs = processor(images=image, return_tensors="pt")
feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
# Step 2: Modify classification head for the new number of classes
num_classes = 10  # Update with your new number of classes
model.classifier = torch.nn.Linear(model.config.hidden_size, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [19]:
selector = ptan.actions.ArgmaxActionSelector()
agent = ptan.agent.PolicyAgent(model=model, action_selector=selector, apply_softmax=True,device=device)


In [99]:
index_to_label = {
    0: 'Airplane',
    1: 'Automobile',
    2: 'Bird',
    3: 'Cat',
    4: 'Deer',
    5: 'Dog',
    6: 'Frog',
    7: 'Horse',
    8: 'Ship',
    9: 'Truck'
}

In [21]:
import gym
import gym.spaces
from gym.utils import seeding
from gym.envs.registration import EnvSpec
import enum
import numpy as np

In [41]:
class Actions(enum.Enum):
    Airplane = 0
    Automobile = 1
    Bird = 2
    Cat = 3
    Deer = 4
    Dog = 5
    Frog = 6
    Horse = 7
    Ship = 8
    Truck = 9

# action_string = 'Deer'
# if hasattr(Actions, action_string):
#     action_enum_member = getattr(Actions, action_string)
#     print(action_enum_member)
#     print(isinstance(action_enum_member, Actions))  # This will print True
# else:
#     print(f"{action_string} is not a valid member of Actions enum.")

In [132]:
class state:
   def __init__(self,commission_prec,reward_perc):
      self.commission_prec=commission_prec
      self.reward_perc = reward_perc
   def step(self,action,label):
      reward = 0.0
      done = False
      if label is not None:
         if hasattr(Actions, label):
            label_enum_member = getattr(Actions, label)
            if label_enum_member is action:
               reward += 100.0 * (1.0 / self.commission_prec - 1.0)
               done = True
            else:
               reward -= self.reward_perc
               done = False
         return self.observation,reward, done
      else:
          done = True
          return self.observation,reward, done
   def update(self, new_observation):
        # Update the state with the new observation
        self.observation = new_observation
   def reset(self,observation):
       self.observation = observation

In [140]:
class CIFAR10Env(gym.Env):
    metadata = {'render.modes': ['human']}
    spec = EnvSpec("CIFAR10Env-v0",entry_point='CIFAR10Env')
    def __init__(self,net ):
        super(CIFAR10Env, self).__init__()
        self.observation_space = gym.spaces.Box(
            low=0, high=255,shape=(224, 224, 3), dtype=np.uint8)
        self.net = net
        self.action_space = gym.spaces.Discrete(n=len(Actions))
        self.state = state(0.7,0.3)


    def reset(self):
        # Reset the environment
        # Return initial observation
        
        return np.zeros((10,), dtype=np.float32)
    

    def update(self, new_observation_path):
        # Update the state with the new observation
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])

        image_path = new_observation_path
        image = Image.open(image_path)
        image_tensor = transform(image)
        # image_tensor = (image_tensor - image_tensor.min()) / (image_tensor.max() - image_tensor.min())  # Normalize to [0, 1]

        # Step 3: Forward pass
        inputs = image_tensor.unsqueeze(0).to(device)  # Add batch dimension
        self.observation_space = inputs
        self.state.update(self.observation_space)

    @torch.no_grad()
    def step(self, label):
        # Take a step in the environment based on the given action
        # Return new observation, reward, done, and info

        outputs = self.net(self.observation_space)

        # Step 4: Post-process predictions
        # Assuming 'logits' is the key returned by your model
        logits = outputs.logits if hasattr(outputs, "logits") else outputs.last_hidden_state
        probabilities = torch.nn.functional.softmax(logits, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()
        predicted_label = index_to_label[predicted_class]
        print(predicted_label)
        action = getattr(Actions, predicted_label)

        obs,reward, done = self.state.step(action,label)
        info = {
            "obs": obs,
            "reward": reward,
            "done":done
        }
        return obs,reward, done, info,logits
    def set_observation(self,input_frame):
        self.observation_space = input_frame

    def _reset_environment(self):
        # Implement your environment's reset logic here and return the initial observation
        # This is a placeholder, replace it with your own environment's reset logic
        initial_observation = np.random.randint(0, 256, size=(224, 224, 3), dtype=np.uint8)
        return initial_observation


In [141]:
env = CIFAR10Env(model)
env.action_space
env.update('0002.jpg')
print(env.observation_space)
o,r,d,_,loggits=env.step("Bird")

Discrete(10)

In [144]:
print(loggits)

tensor([[ 0.0592,  0.0155, -0.0229, -0.0412, -0.1823, -0.0933,  0.0322,  0.1976,
         -0.0753,  0.2858]], device='cuda:0')
